In [8]:
import pandas as pd
import numpy
import yfinance as yf

In [2]:
# Fetching historical data

In [3]:
# 81/22 8/1/23
tickers = ['JPM','MS','BAC']
data = yf.download(tickers,start='2022-08-01',end='2023-08-01')['Adj Close']

[*********************100%%**********************]  3 of 3 completed


In [4]:
data

Ticker,BAC,JPM,MS
Date,,,
2022-08-01,31.869623,108.613457,78.671318
2022-08-02,31.236202,106.920692,78.147423
2022-08-03,31.803440,108.042870,79.569473
2022-08-04,31.576546,106.854118,79.859489
2022-08-05,32.105972,110.087517,80.579865
...,...,...,...
2023-07-25,31.220678,153.820053,90.690437
2023-07-26,31.473164,154.742050,89.845200
2023-07-27,31.026461,153.035355,89.086403


In [11]:
data_df = pd.DataFrame(data)
data_df.to_csv('data.csv')

In [12]:
# Calculating daily returns (price_today/price_yesterday -1)

In [54]:
returns = pd.DataFrame()
column_headers = data_df.columns.values
for stock in range(data_df.shape[1]):
    ret = []
    for i in range(data_df.shape[0]):
        if i==0:
            continue
        else:
            ret.append((data_df.iloc[i,stock]/data_df.iloc[i-1,stock])-1)
    returns.insert(stock,column_headers[stock],ret)

In [57]:
returns*100

Ticker,BAC,JPM,MS
Date,,,
2022-08-02,-1.987538,-1.558522,-0.665929
2022-08-03,1.815963,1.049542,1.819702
2022-08-04,-0.713427,-1.100259,0.364482
2022-08-05,1.676645,3.025993,0.902053
2022-08-08,-1.383984,-1.218056,-0.116098
...,...,...,...
2023-07-25,-1.531402,-0.746825,-0.264061
2023-07-26,0.808712,0.599400,-0.932003
2023-07-27,-1.419314,-1.102929,-0.844560


In [ ]:
# Assuming weights

In [93]:
total_portfolio = 1000000
w = [0.03,0.03,0.04]
weights = pd.DataFrame(list(zip([30],[30],[40])),columns = ['BAC','JPM','MS'])
value = weights*total_portfolio

In [ ]:
# Calculating portfolio mean

In [ ]:
ans = 0
for i in range(data_df.shape[1]):
    ans+=()

In [222]:
mean_df = returns.dot(w)
mean_df

Date
2022-08-02   -0.001330
2022-08-03    0.001588
2022-08-04   -0.000398
2022-08-05    0.001772
2022-08-08   -0.000827
                ...   
2023-07-25   -0.000789
2023-07-26    0.000050
2023-07-27   -0.001094
2023-07-28   -0.000019
2023-07-31    0.000290
Length: 250, dtype: float64

In [223]:
portfolio_mean = mean_df.mean()

In [ ]:
# Calculating the covariance Matrix

In [102]:
# correlation between columns
corr_df = returns.corr()
corr_df

Ticker,BAC,JPM,MS
Ticker,,,
BAC,1.000000,0.819798,0.752721
JPM,0.819798,1.000000,0.664382
MS,0.752721,0.664382,1.000000


In [192]:
# cov(a,b) = corr(a,b) * sd(a) * sd(b)
# standard deviation of returns
sd_df = returns.std()

In [261]:
sd_df

Ticker
BAC    0.018240
JPM    0.016283
MS     0.017193
dtype: float64

In [268]:
cov_matrix = pd.DataFrame()
for i in range(data_df.shape[1]):
    ret = []
    for j in range(data_df.shape[1]):
        ret.append(corr_df.iloc[j,i]*sd_df[i]*sd_df[j])
    cov_matrix.insert(i,i,ret)

In [269]:
cov_matrix = cov_matrix
cov_matrix

,0,1,2
0,0.000333,0.000243,0.000236
1,0.000243,0.000265,0.000186
2,0.000236,0.000186,0.000296


In [160]:
# calculating portfolio SD

In [293]:
portfolio_variance = (weights).mul(cov_matrix).mul(weights.transpose())
portfolio_variance = portfolio_variance[0][0]

In [294]:
import math
portfolio_sd = math.sqrt(portfolio_variance)

In [295]:
portfolio_sd,portfolio_mean

(0.5471945168609231, 7.370827764365035e-05)

In [296]:
from scipy.stats import norm
def parametric_var(mean, std_dev,confidence_level):
    z_score = norm.ppf(1 - confidence_level)
    return mean + z_score * std_dev

In [297]:
print("1 Day VaR at confidence score = 0.90: ",parametric_var(portfolio_mean/100, 1.56/100,0.90)*total_portfolio)
print("1 Day VaR at confidence score = 0.95 :",parametric_var(portfolio_mean/100, 1.56/100,0.95)*total_portfolio)
print("1 Day VaR at confidence score = 0.99",parametric_var(portfolio_mean/100, 1.56/100,0.99)*total_portfolio)

1 Day VaR at confidence score = 0.90:  -19991.467339719333
1 Day VaR at confidence score = 0.95 : -25658.979497666533
1 Day VaR at confidence score = 0.99 -36290.28975226068
